In [108]:
import math
import numpy as np
import pandas as pd
from collections import Counter

In [109]:
import warnings
warnings.filterwarnings(action='once')

In [111]:
np.random.seed(seed=408)

In [196]:
# Create df with team name, wins, losses, elo
team_df = pd.read_csv("C:\\Users\\Divya Parmar's PC\\Documents\\NBA-Predictions-2020-Restart\\nba_elo_data.csv")
original_team_object = pd.read_csv("C:\\Users\\Divya Parmar's PC\\Documents\\NBA-Predictions-2020-Restart\\nba_elo_data.csv")

In [175]:
# Create weighed elo value to estimate team's true ability
# Weigh between current elo value and potential full strength elo
# Use 0.5 meaning we give half credit to current and full strength
elo_weight_start_season = 0.5

In [176]:
# Officially create weighed elo value to estimate team's true ability
team_df['elo_value'] = elo_weight_start_season * team_df['elo_basic'] + (1 - elo_weight_start_season) * team_df['elo_playoff_full']

In [177]:
team_df['elo_list'] = [[x] for x in team_df.elo_value]

In [89]:
#Create copies of dataframe
team_df_testing = team_df
team_df_backup = team_df

In [11]:
#Bring in the schedule
schedule = pd.read_csv("C:\\Users\\Divya Parmar's PC\\Documents\\NBA-Predictions-2020-Restart\\nba_regular_season_games_2020.csv")

In [178]:
# Look at our dataframe
team_df

,team_abbrev,team_nickname,elo_basic,elo_playoff_full,wins,losses,conference,elo_value,elo_list
0,LAL,Lakers,1678,1710,49,14,western,1694.0,[1694.0]
1,LAC,Clippers,1636,1732,44,20,western,1684.0,[1684.0]
2,MIL,Bucks,1716,1694,53,12,eastern,1705.0,[1705.0]
3,PHI,76ers,1564,1691,39,26,eastern,1627.5,[1627.5]
4,HOU,Rockets,1565,1701,40,24,western,1633.0,[1633.0]
5,BOS,Celtics,1628,1682,43,21,eastern,1655.0,[1655.0]
6,TOR,Raptors,1679,1628,46,18,eastern,1653.5,[1653.5]
7,DEN,Nuggets,1579,1638,43,22,western,1608.5,[1608.5]
8,OKC,Thunder,1608,1583,40,24,western,1595.5,[1595.5]
9,DAL,Mavericks,1584,1643,40,27,western,1613.5,[1613.5]


In [19]:
# Look at schedule
schedule

,team_a,team_b
0,UTA,NOP
1,LAC,LAL
2,ORL,BKN
3,ORL,BKN
4,MEM,POR
5,PHX,WAS
6,BOS,MIL
7,SAC,SAS
8,HOU,DAL
9,MIA,DEN


In [20]:
# First let's write a function to calculate elo
# Thankfully it's  already been done
# https://www.geeksforgeeks.org/elo-rating-algorithm/

In [26]:
def elo_probability(rating1, rating2):
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * (rating2 - rating1)/400))

In [27]:
# Validate that the probabilities add up to 1 for a given matchup
elo_probability(2000, 1200) + elo_probability(1200, 2000)

1.0

In [28]:
elo_probability(1200, 2000)

0.009900990099009901

In [29]:
# K-Factor determines how quickly elo score reacts to new events
# FiveThirtyEight recommends k-factor of 20 for NBA games
k = 20

In [30]:
def game_winner(rating1, rating2):
    prob = elo_probability(rating1, rating2)
    random_value = np.random.uniform(0,1)
    if random_value <= prob:
        return 1
    else:
        return 2

In [31]:
# Unit test of game_winner function
game_winner(1800, 1700)

1

In [32]:
def elo_update(rating_a, rating_b):
    Pa = elo_probability(rating_a, rating_b)
    Pb = elo_probability(rating_b, rating_a)
    
    game_winner_output = game_winner(rating_a, rating_b)
    
    if game_winner_output == 1:
        rating_a_updated = rating_a + 20 * (1-Pa)
        rating_b_updated = rating_b + 20 * (0-Pb)
    
    elif game_winner_output == 2:
        rating_a_updated = rating_a + 20 * (0-Pa)
        rating_b_updated = rating_b + 20 * (1-Pb)
    
    else:
        rating_a_updated = rating_a
        rating_b_updated = rating_b
    return rating_a_updated, rating_b_updated, game_winner_output

In [33]:
# Unit test elo_update
test_elo_a, test_elo_b, winner = elo_update(1800, 1700)
print(test_elo_a, test_elo_b, winner)

(1787.1987000039423, 1712.8012999960577, 2)


In [170]:
def game_simulation_regular_season(team_a, team_b, df):
    team_a = team_a
    team_b = team_b
    input_df = df
    ###
    #Get elo values from database
    ###
    elo_a = input_df['elo_list'][input_df['team_abbrev'] == team_a].values[0][-1]
    #print("elo a", elo_a)
    
    elo_b = input_df['elo_list'][input_df['team_abbrev'] == team_b].values[0][-1]
    #print("elo b", elo_b)
    
    # Run elo update and game winner functions
    new_elo_a, new_elo_b, game_winner_output = elo_update(elo_a, elo_b)
    
    # Update elo for first team
    input_df['elo_list'][input_df['team_abbrev'] == team_a].values[0].append(new_elo_a)
    
    # Update elo for second team
    input_df['elo_list'][input_df['team_abbrev'] == team_b].values[0].append(new_elo_b)
    
    # Add wins and losses to respective teams
    if game_winner_output == 1:
        input_df['wins'][input_df['team_abbrev'] == team_a] += 1
        input_df['losses'][input_df['team_abbrev'] == team_b] += 1
    elif game_winner_output == 2:
        input_df['wins'][input_df['team_abbrev'] == team_b] += 1
        input_df['losses'][input_df['team_abbrev'] == team_a] += 1
    else:
        pass
    return input_df

In [35]:
team_df_testing['elo_list'][team_df_testing['team_abbrev'] == 'LAL'].values[-1][-1]

1694.0

In [36]:
# Unit test game_simulation function
game_simulation_regular_season('LAL', 'LAC', team_df_testing)

C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,team_abbrev,team_nickname,elo_basic,elo_playoff_full,wins,losses,conference,elo_value,elo_list
0,LAL,Lakers,1678,1710,49,15,western,1694.0,"[1694.0, 1683.71225632]"
1,LAC,Clippers,1636,1732,45,20,western,1684.0,"[1684.0, 1694.28774368]"
2,MIL,Bucks,1716,1694,53,12,eastern,1705.0,[1705.0]
3,PHI,76ers,1564,1691,39,26,eastern,1627.5,[1627.5]
4,HOU,Rockets,1565,1701,40,24,western,1633.0,[1633.0]
5,BOS,Celtics,1628,1682,43,21,eastern,1655.0,[1655.0]
6,TOR,Raptors,1679,1628,46,18,eastern,1653.5,[1653.5]
7,DEN,Nuggets,1579,1638,43,22,western,1608.5,[1608.5]
8,OKC,Thunder,1608,1583,40,24,western,1595.5,[1595.5]
9,DAL,Mavericks,1584,1643,40,27,western,1613.5,[1613.5]


In [45]:
team_df_testing

,team_abbrev,team_nickname,elo_basic,elo_playoff_full,wins,losses,conference,elo_value,elo_list,winning_percentage,elo_final_regular_season
0,LAL,Lakers,1678,1710,56,16,western,1694.0,"[1694.0, 1683.71225632, 1674.01654934, 1683.42...",0.777778,1724.181604
1,LAC,Clippers,1636,1732,51,22,western,1684.0,"[1684.0, 1694.28774368, 1703.98345066, 1710.22...",0.698630,1703.862966
2,MIL,Bucks,1716,1694,58,15,eastern,1705.0,"[1705.0, 1713.57073765, 1721.55154454, 1726.46...",0.794521,1693.291963
3,PHI,76ers,1564,1691,45,28,eastern,1627.5,"[1627.5, 1615.94410457, 1602.38578024, 1607.44...",0.616438,1650.901135
4,HOU,Rockets,1565,1701,43,29,western,1633.0,"[1633.0, 1642.43933347, 1634.45852658, 1641.58...",0.597222,1604.060336
5,BOS,Celtics,1628,1682,46,26,eastern,1655.0,"[1655.0, 1646.42926235, 1653.42639576, 1639.97...",0.638889,1613.110102
6,TOR,Raptors,1679,1628,51,21,eastern,1653.5,"[1653.5, 1644.08982833, 1651.05836852, 1637.55...",0.708333,1662.524108
7,DEN,Nuggets,1579,1638,46,27,western,1608.5,"[1608.5, 1616.65067539, 1625.75108174, 1632.28...",0.630137,1586.534914
8,OKC,Thunder,1608,1583,43,29,western,1595.5,"[1595.5, 1585.3108665, 1576.21046015, 1569.400...",0.597222,1574.761197
9,DAL,Mavericks,1584,1643,42,33,western,1613.5,"[1613.5, 1604.06066653, 1591.01899343, 1578.20...",0.560000,1575.261187


In [38]:
# for index, row in schedule.iterrows():
#     game_simulation_regular_season(row['team_a'], row['team_b'], team_df_testing)

C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [47]:
# def simulate_regular_season(input_df, schedule_df):
#     for index, row in schedule_df.iterrows():
#         game_simulation_regular_season(row['team_a'], row['team_b'], input_df)
#     input_df['elo_final_regular_season'] = [x[-1] for x in input_df['elo_list']]
#     input_df['winning_percentage'] = input_df['wins']/(input_df['wins']+input_df['losses'])
#     input_df['elo_into_playoffs'] = 0.5 * input_df['elo_value'] + 0.5 * input_df['elo_final_regular_season']
#     return input_df

In [41]:
# team_df_testing['elo_final_regular_season'] = [x[-1] for x in team_df_testing['elo_list']]

In [39]:
# team_df_testing['winning_percentage'] = team_df_testing['wins']/(team_df_testing['wins']+team_df_testing['losses'])

In [48]:
# team_df_testing2 = team_df_backup
# simulate_regular_season(team_df_testing2, schedule)

C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,team_abbrev,team_nickname,elo_basic,elo_playoff_full,wins,losses,conference,elo_value,elo_list,winning_percentage,elo_final_regular_season,elo_into_playoffs
0,LAL,Lakers,1678,1710,61,19,western,1694.0,"[1694.0, 1683.71225632, 1674.01654934, 1683.42...",0.762500,1719.421207,1706.710604
1,LAC,Clippers,1636,1732,57,24,western,1684.0,"[1684.0, 1694.28774368, 1703.98345066, 1710.22...",0.703704,1713.082893,1698.541447
2,MIL,Bucks,1716,1694,63,18,eastern,1705.0,"[1705.0, 1713.57073765, 1721.55154454, 1726.46...",0.777778,1686.373044,1695.686522
3,PHI,76ers,1564,1691,51,30,eastern,1627.5,"[1627.5, 1615.94410457, 1602.38578024, 1607.44...",0.629630,1663.398536,1645.449268
4,HOU,Rockets,1565,1701,47,33,western,1633.0,"[1633.0, 1642.43933347, 1634.45852658, 1641.58...",0.587500,1600.920257,1616.960128
5,BOS,Celtics,1628,1682,50,30,eastern,1655.0,"[1655.0, 1646.42926235, 1653.42639576, 1639.97...",0.625000,1602.235935,1628.617967
6,TOR,Raptors,1679,1628,56,24,eastern,1653.5,"[1653.5, 1644.08982833, 1651.05836852, 1637.55...",0.700000,1670.647038,1662.073519
7,DEN,Nuggets,1579,1638,50,31,western,1608.5,"[1608.5, 1616.65067539, 1625.75108174, 1632.28...",0.617284,1588.134191,1598.317095
8,OKC,Thunder,1608,1583,48,32,western,1595.5,"[1595.5, 1585.3108665, 1576.21046015, 1569.400...",0.600000,1600.173230,1597.836615
9,DAL,Mavericks,1584,1643,46,37,western,1613.5,"[1613.5, 1604.06066653, 1591.01899343, 1578.20...",0.554217,1575.369462,1594.434731


In [118]:
# Two dimensional dictionary of team and seed
# Sum it up

regular_season_standings = {'team': [], 'seed':[], 'conference':[]}

#pd.DataFrame({'team':[],'seed_1':[], 'seed_2':[], 'seed_3':[], 'seed_4':[], 'seed_5':[], 'seed_6':[], 'seed_7':[], 'seed_8':[],})

In [144]:
# Define function to simulate all regular season games and return a final dataframe at the end of the regular season
def simulate_regular_season(df, schedule_df):
    input_df = df
    # Simulate the season
    for index, row in schedule_df.iterrows():
        game_simulation_regular_season(row['team_a'], row['team_b'], input_df)

    # Create certain column that will be needed later on
    input_df['elo_final_regular_season'] = [x[-1] for x in input_df['elo_list']]
    input_df['winning_percentage'] = input_df['wins']/(input_df['wins']+input_df['losses'])
    input_df['elo_into_playoffs'] = 0.5 * input_df['elo_value'] + 0.5 * input_df['elo_final_regular_season']

    # Create western conference and eastern conference dataframes
    western_df = input_df[input_df.conference == 'western'].sort_values(by=['winning_percentage', 'elo_final_regular_season'], ascending=False)
    western_df['seed'] = [x for x in range(1, len(western_df.team_abbrev)+1)]
    eastern_df = input_df[input_df.conference == 'eastern'].sort_values(by=['winning_percentage', 'elo_final_regular_season'], ascending=False)
    eastern_df['seed'] = [x for x in range(1, len(eastern_df.team_abbrev)+1)]

    return input_df, western_df, eastern_df

In [116]:
team_df_testing3, west_df_test, east_df_test = simulate_regular_season(team_df, schedule)

C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [117]:
west_df_test

,team_abbrev,team_nickname,elo_basic,elo_playoff_full,wins,losses,conference,elo_value,elo_list,elo_final_regular_season,winning_percentage,elo_into_playoffs,seed
0,LAL,Lakers,1678,1710,53,18,western,1694.0,"[1694.0, 1703.71225632, 1712.27701006, 1698.62...",1673.714970,0.746479,1683.857485,1
1,LAC,Clippers,1636,1732,49,23,western,1684.0,"[1684.0, 1674.28774368, 1661.28309103, 1647.13...",1685.078701,0.680556,1684.539350,2
7,DEN,Nuggets,1579,1638,48,25,western,1608.5,"[1608.5, 1616.65067539, 1626.3244053, 1633.363...",1623.368967,0.657534,1615.934484,3
11,UTA,Jazz,1574,1584,46,26,western,1579.0,"[1579.0, 1588.92804546, 1579.11717896, 1592.77...",1595.968332,0.638889,1587.484166,4
4,HOU,Rockets,1565,1701,44,28,western,1633.0,"[1633.0, 1642.43933347, 1634.45852658, 1621.08...",1626.161302,0.611111,1629.580651,5
8,OKC,Thunder,1608,1583,44,28,western,1595.5,"[1595.5, 1605.3108665, 1595.63713659, 1588.517...",1590.173562,0.611111,1592.836781,6
9,DAL,Mavericks,1584,1643,42,33,western,1613.5,"[1613.5, 1604.06066653, 1591.01899343, 1598.72...",1570.273225,0.560000,1591.886612,7
17,MEM,Grizzlies,1553,1456,34,39,western,1504.5,"[1504.5, 1515.21831854, 1504.68593745, 1496.80...",1487.429616,0.465753,1495.964808,8
19,SAS,Spurs,1499,1473,32,39,western,1486.0,"[1486.0, 1496.70400017, 1507.23638125, 1520.26...",1522.283021,0.450704,1504.141511,9
14,POR,Trailblazers,1490,1569,33,41,western,1529.5,"[1529.5, 1518.78168146, 1512.29882936, 1525.67...",1545.430895,0.445946,1537.465447,10


In [81]:
east_df_test.sort_values(by=['winning_percentage'], ascending=False)
east_df_test.reset_index()
east_df_test['level_0'] = east_df_test.index
east_df_test

C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,index,team_abbrev,team_nickname,elo_basic,elo_playoff_full,wins,losses,conference,elo_value,elo_list,winning_percentage,elo_final_regular_season,elo_into_playoffs,level_0
2,2,MIL,Bucks,1716,1694,144,25,eastern,1705.0,"[1705.0, 1713.57073765, 1721.55154454, 1726.46...",0.852071,1866.141887,1785.570944,2
3,3,PHI,76ers,1564,1691,117,52,eastern,1627.5,"[1627.5, 1615.94410457, 1602.38578024, 1607.44...",0.692308,1721.426979,1674.463490,3
5,5,BOS,Celtics,1628,1682,95,73,eastern,1655.0,"[1655.0, 1646.42926235, 1653.42639576, 1639.97...",0.565476,1589.249632,1622.124816,5
6,6,TOR,Raptors,1679,1628,113,55,eastern,1653.5,"[1653.5, 1644.08982833, 1651.05836852, 1637.55...",0.672619,1747.769808,1700.634904,6
12,12,MIA,Heat,1553,1534,78,91,eastern,1543.5,"[1543.5, 1535.34932461, 1528.38078442, 1541.83...",0.461538,1458.316544,1500.908272,12
13,13,IND,Pacers,1560,1586,91,78,eastern,1573.0,"[1573.0, 1584.55589543, 1569.67614581, 1578.62...",0.538462,1540.566727,1556.783364,13
15,15,ORL,Magic,1503,1546,61,121,eastern,1524.5,"[1524.5, 1534.3417105, 1523.61814012, 1532.933...",0.335165,1340.494949,1432.497474,15
16,16,BKN,Nets,1478,1560,93,88,eastern,1519.0,"[1519.0, 1509.1582895, 1519.88185988, 1526.718...",0.513812,1586.285019,1552.642510,16
20,20,WAS,Wizards,1396,1432,51,117,eastern,1414.0,"[1414.0, 1406.07072637, 1399.2341256, 1414.113...",0.303571,1359.711898,1386.855949,20


In [94]:
for i in ['eastern', 'western']:
    for index, row in team_df_testing3[team_df_testing3.conference == i].sort_values(by=['winning_percentage', 'elo_final_regular_season'], ascending=False).reset_index().reset_index().iterrows():
        regular_season_standings['team'].append(row['team_abbrev'])
        regular_season_standings['conference'].append(row['conference'])
        regular_season_standings['seed'].append(int(row['level_0'] + 1))
        # When looping through season simulations do this:
        # regular_season_standings['iteration'].append(i)

ValueError: cannot insert level_0, already exists

In [48]:
standings_df = pd.DataFrame(regular_season_standings)
standings_df

,conference,seed,team
0,eastern,1,MIL
1,eastern,2,PHI
2,eastern,3,BOS
3,eastern,4,TOR
4,eastern,5,MIA
5,eastern,6,IND
6,eastern,7,ORL
7,eastern,8,BKN
8,eastern,9,WAS
9,western,1,LAL


In [53]:
pd.pivot_table(standings_df[standings_df.conference == 'western'], values='conference', index=['team'], columns=['seed'], aggfunc=np.count_nonzero)

#Add this sort
# .sort_values(by=2012, ascending=False).index)

seed,1,2,3,4,5,6,7,8,9,10,11,12,13
team,,,,,,,,,,,,,
DAL,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DEN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HOU,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LAC,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LAL,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
NOP,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
OKC,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PHX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [ ]:
i = 1
while i < 500:
    # Reset team dataframe with wins/losses and standings
    # Set schedule
    
    # Play regular season games and update team dataframe
    # Append regular_season_standings dataframe with output (will aggregate at the end)
    
    # Do play-in round and get 8 playoff teams (write this to some object)
    # LOW PRIORITY - Append dataframe with list of playoff teams and their seeds (will aggregate at the end)
    
    # Simulate playoffs and append each playoff round to the end of a column (do list comprehension)
    # Take last round made and save that as a variable (do list comphrehension) - second round, conf finals, finals, champion


# Now outside of loop
# Aggregate regular_season_standings
# Aggregate playoff round made and create pandas pivot_table (don't worry about sorting for now)
    

In [39]:
team_df_testing[team_df_testing.conference == 'eastern'].reset_index().reset_index()

,level_0,index,team_abbrev,team_nickname,elo_basic,elo_playoff_full,wins,losses,conference,elo_value,elo_list,elo_final_regular_season
0,0,2,MIL,Bucks,1716,1694,58,15,eastern,1705.0,"[1705.0, 1713.57073765, 1721.55154454, 1726.46...",1693.291963
1,1,3,PHI,76ers,1564,1691,45,28,eastern,1627.5,"[1627.5, 1615.94410457, 1602.38578024, 1607.44...",1650.901135
2,2,5,BOS,Celtics,1628,1682,46,26,eastern,1655.0,"[1655.0, 1646.42926235, 1653.42639576, 1639.97...",1613.110102
3,3,6,TOR,Raptors,1679,1628,51,21,eastern,1653.5,"[1653.5, 1644.08982833, 1651.05836852, 1637.55...",1662.524108
4,4,12,MIA,Heat,1553,1534,47,26,eastern,1543.5,"[1543.5, 1535.34932461, 1528.38078442, 1541.83...",1594.567978
5,5,13,IND,Pacers,1560,1586,42,31,eastern,1573.0,"[1573.0, 1584.55589543, 1569.67614581, 1578.62...",1553.228435
6,6,15,ORL,Magic,1503,1546,33,41,eastern,1524.5,"[1524.5, 1534.3417105, 1523.61814012, 1532.933...",1503.667754
7,7,16,BKN,Nets,1478,1560,34,39,eastern,1519.0,"[1519.0, 1509.1582895, 1519.88185988, 1526.718...",1521.091977
8,8,20,WAS,Wizards,1396,1432,26,46,eastern,1414.0,"[1414.0, 1406.07072637, 1399.2341256, 1414.113...",1411.124649


In [57]:
team_df_testing['elo_into_playoffs'] = 0.5 * team_df_testing['elo_value'] + 0.5 * team_df_testing['elo_final_regular_season']

In [45]:
# Create win percentage column
team_df['win_percentage'] = team_df['wins']/(team_df['wins']+team_df['losses'])

In [46]:
# Create conference specific dataframes
df_western = team_df[team_df['conference'] == 'western']
df_eastern = team_df[team_df['conference'] == 'eastern']

In [47]:
# Get Western standings at the end of the regular season
df_western['standing_min'] = df_western['win_percentage'].rank(method='min', ascending=False)
df_western['standing_dense'] = df_western['win_percentage'].rank(method='dense', ascending=False)
df_western['standing_random'] = df_western['win_percentage'].rank(method='first', ascending=False)
df_western.sort_values(by=['standing_min', 'standing_random'], inplace=True)
df_western.reset_index(inplace=True)
df_western['ahead_wins'] = df_western['wins'].shift(1)
df_western['ahead_losses'] = df_western['losses'].shift(1)

df_western['games_back_of_ahead_team'] = ((df_western['ahead_wins'] - df_western['wins']) + (df_western['losses'] - df_western['ahead_losses']))/2

C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [48]:
df_western

,index,team_abbrev,team_nickname,elo_basic,elo_playoff_full,wins,losses,conference,elo_value,elo_list,elo_final_regular_season,win_percentage,standing_min,standing_dense,standing_random,ahead_wins,ahead_losses,games_back_of_ahead_team
0,0,LAL,Lakers,1678,1710,56,16,western,1694.0,"[1694.0, 1683.71225632, 1674.01654934, 1683.42...",1724.181604,0.777778,1.0,1.0,1.0,NaN,NaN,NaN
1,1,LAC,Clippers,1636,1732,51,22,western,1684.0,"[1684.0, 1694.28774368, 1703.98345066, 1710.22...",1703.862966,0.698630,2.0,2.0,2.0,56.0,16.0,5.5
2,11,UTA,Jazz,1574,1584,48,24,western,1579.0,"[1579.0, 1588.92804546, 1599.11717896, 1591.49...",1631.234866,0.666667,3.0,3.0,3.0,51.0,22.0,2.5
3,7,DEN,Nuggets,1579,1638,46,27,western,1608.5,"[1608.5, 1616.65067539, 1625.75108174, 1632.28...",1586.534914,0.630137,4.0,4.0,4.0,48.0,24.0,2.5
4,4,HOU,Rockets,1565,1701,43,29,western,1633.0,"[1633.0, 1642.43933347, 1634.45852658, 1641.58...",1604.060336,0.597222,5.0,5.0,5.0,46.0,27.0,2.5
5,8,OKC,Thunder,1608,1583,43,29,western,1595.5,"[1595.5, 1585.3108665, 1576.21046015, 1569.400...",1574.761197,0.597222,5.0,5.0,6.0,43.0,29.0,0.0
6,9,DAL,Mavericks,1584,1643,42,33,western,1613.5,"[1613.5, 1604.06066653, 1591.01899343, 1578.20...",1575.261187,0.560000,7.0,6.0,7.0,43.0,29.0,2.5
7,17,MEM,Grizzlies,1553,1456,35,38,western,1504.5,"[1504.5, 1495.21831854, 1505.26107963, 1496.83...",1506.238907,0.479452,8.0,7.0,8.0,42.0,33.0,6.0
8,10,NOP,Pelicans,1542,1611,33,39,western,1576.5,"[1576.5, 1566.57195454, 1560.33289258, 1568.76...",1585.315911,0.458333,9.0,8.0,9.0,35.0,38.0,1.5
9,14,POR,Trailblazers,1490,1569,32,42,western,1529.5,"[1529.5, 1538.78168146, 1531.78454805, 1524.65...",1523.966998,0.432432,10.0,9.0,10.0,33.0,39.0,2.0


In [49]:
# Get Eastern standings at the end of the regular season
df_eastern['standing_min'] = df_eastern['win_percentage'].rank(method='min', ascending=False)
df_eastern['standing_dense'] = df_eastern['win_percentage'].rank(method='dense', ascending=False)
df_eastern['standing_random'] = df_eastern['win_percentage'].rank(method='first', ascending=False)
df_eastern.sort_values(by=['standing_min', 'standing_random'], inplace=True)
df_eastern.reset_index(inplace=True)
df_eastern['ahead_wins'] = df_eastern['wins'].shift(1)
df_eastern['ahead_losses'] = df_eastern['losses'].shift(1)

df_eastern['games_back_of_ahead_team'] = ((df_eastern['ahead_wins'] - df_eastern['wins']) + (df_eastern['losses'] - df_eastern['ahead_losses']))/2

C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [50]:
def bracket_cleaner(input_df):
    input_df['standing_min'] = input_df['win_percentage'].rank(method='min', ascending=False)
    input_df['standing_dense'] = input_df['win_percentage'].rank(method='dense', ascending=False)
    input_df['standing_random'] = input_df['win_percentage'].rank(method='first', ascending=False)
    input_df.sort_values(by=['standing_min', 'standing_random'], inplace=True)
    input_df.reset_index(inplace=True)
    input_df['ahead_wins'] = input_df['wins'].shift(1)
    input_df['ahead_losses'] = input_df['losses'].shift(1)
    input_df['games_back_of_ahead_team'] = ((input_df['ahead_wins'] - input_df['wins']) + (input_df['losses'] - input_df['ahead_losses']))/2
    return input_df

In [51]:
# Create series for each Western seed 1-7 and Eastern seed 1-7
west_seed_one = df_western.loc[0]
west_seed_two = df_western.loc[1]
west_seed_three = df_western.loc[2]
west_seed_four = df_western.loc[3]
west_seed_five = df_western.loc[4]
west_seed_six = df_western.loc[5]
west_seed_seven = df_western.loc[6]
east_seed_one = df_eastern.loc[0]
east_seed_two = df_eastern.loc[1]
east_seed_three = df_eastern.loc[2]
east_seed_four = df_eastern.loc[3]
east_seed_five = df_eastern.loc[4]
east_seed_six = df_eastern.loc[5]
east_seed_seven = df_eastern.loc[6]

In [52]:
# # Play-in round function
# def play_in_round(input_df):
#     eight = input_df.loc[7]
#     nine = input_df.loc[8]
#     if nine['games_back_of_ahead_team'] > 4:
#         return eight
#     elif nine['games_back_of_ahead_team'] <= 4:
#         rating1_1, rating2_1, outcome1 = elo_update(eight['elo_value'], nine['elo_value'])
#         eight['elo_value'] = rating1_1
#         nine['elo_value'] = rating2_1
#         if outcome1 == 1:
#             return eight
#         else:
#             rating1_2, rating2_2, outcome2 = elo_update(eight['elo_value'], nine['elo_value'])
#             eight['elo_value'] = rating1_2
#             nine['elo_value'] = rating2_2
#             if outcome2 == 1:
#                 return eight
#             else:
#                 return nine

In [53]:
# Unit test play in round function
df_western_test = df_western
final_west_team = play_in_round(df_western_test)
print(final_west_team)

index                                                                      17
team_abbrev                                                               MEM
team_nickname                                                       Grizzlies
elo_basic                                                                1553
elo_playoff_full                                                         1456
wins                                                                       35
losses                                                                     38
conference                                                            western
elo_value                                                             1509.02
elo_list                    [1504.5, 1495.21831854, 1505.26107963, 1496.83...
elo_final_regular_season                                              1506.24
win_percentage                                                       0.479452
standing_min                                                    

C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramFiles\Anaconda2

In [54]:
# Assign final playoff spots
west_seed_eight = play_in_round(df_western)

C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramFiles\Anaconda2

In [55]:
west_seed_eight

index                                                                      10
team_abbrev                                                               NOP
team_nickname                                                        Pelicans
elo_basic                                                                1542
elo_playoff_full                                                         1611
wins                                                                       33
losses                                                                     39
conference                                                            western
elo_value                                                             1591.98
elo_list                    [1576.5, 1566.57195454, 1560.33289258, 1568.76...
elo_final_regular_season                                              1585.32
win_percentage                                                       0.458333
standing_min                                                    

In [56]:
df_eastern

,index,team_abbrev,team_nickname,elo_basic,elo_playoff_full,wins,losses,conference,elo_value,elo_list,elo_final_regular_season,win_percentage,standing_min,standing_dense,standing_random,ahead_wins,ahead_losses,games_back_of_ahead_team
0,2,MIL,Bucks,1716,1694,58,15,eastern,1705.0,"[1705.0, 1713.57073765, 1721.55154454, 1726.46...",1693.291963,0.794521,1.0,1.0,1.0,NaN,NaN,NaN
1,6,TOR,Raptors,1679,1628,51,21,eastern,1653.5,"[1653.5, 1644.08982833, 1651.05836852, 1637.55...",1662.524108,0.708333,2.0,2.0,2.0,58.0,15.0,6.5
2,12,MIA,Heat,1553,1534,47,26,eastern,1543.5,"[1543.5, 1535.34932461, 1528.38078442, 1541.83...",1594.567978,0.643836,3.0,3.0,3.0,51.0,21.0,4.5
3,5,BOS,Celtics,1628,1682,46,26,eastern,1655.0,"[1655.0, 1646.42926235, 1653.42639576, 1639.97...",1613.110102,0.638889,4.0,4.0,4.0,47.0,26.0,0.5
4,3,PHI,76ers,1564,1691,45,28,eastern,1627.5,"[1627.5, 1615.94410457, 1602.38578024, 1607.44...",1650.901135,0.616438,5.0,5.0,5.0,46.0,26.0,1.5
5,13,IND,Pacers,1560,1586,42,31,eastern,1573.0,"[1573.0, 1584.55589543, 1569.67614581, 1578.62...",1553.228435,0.575342,6.0,6.0,6.0,45.0,28.0,3.0
6,16,BKN,Nets,1478,1560,34,39,eastern,1519.0,"[1519.0, 1509.1582895, 1519.88185988, 1526.718...",1521.091977,0.465753,7.0,7.0,7.0,42.0,31.0,8.0
7,15,ORL,Magic,1503,1546,33,41,eastern,1524.5,"[1524.5, 1534.3417105, 1523.61814012, 1532.933...",1503.667754,0.445946,8.0,8.0,8.0,34.0,39.0,1.5
8,20,WAS,Wizards,1396,1432,26,46,eastern,1414.0,"[1414.0, 1406.07072637, 1399.2341256, 1414.113...",1411.124649,0.361111,9.0,9.0,9.0,33.0,41.0,6.0


In [57]:
east_seed_eight = play_in_round(df_eastern)

In [58]:
# #Simulate playoff series
# def playoff_series_sim(team_one, team_two):
#     team1_wins = 0
#     team2_wins = 0
#     while team1_wins < 4 and team2_wins < 4:
#         rating1, rating2, outcome = elo_update(team_one['elo_value'], team_two['elo_value'])
#         team_one['elo_value'] = rating1
#         team_two['elo_value'] = rating2
#         if outcome == 1:
#             team1_wins += 1
#         elif outcome == 2:
#             team2_wins += 1
#     if team1_wins == 4:
#         return team_one
#     elif team2_wins == 4:
#         return team_two
#     else:
#         return None

In [59]:
winnerofseries = playoff_series_sim(west_seed_eight, east_seed_eight)

C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [60]:
winnerofseries

index                                                                      15
team_abbrev                                                               ORL
team_nickname                                                           Magic
elo_basic                                                                1503
elo_playoff_full                                                         1546
wins                                                                       33
losses                                                                     41
conference                                                            eastern
elo_value                                                             1561.87
elo_list                    [1524.5, 1534.3417105, 1523.61814012, 1532.933...
elo_final_regular_season                                              1503.67
win_percentage                                                       0.445946
standing_min                                                    

In [61]:
#Western playoffs
west18 = playoff_series_sim(west_seed_one, west_seed_eight)
west27 = playoff_series_sim(west_seed_two, west_seed_seven)
west36 = playoff_series_sim(west_seed_three, west_seed_six)
west45 = playoff_series_sim(west_seed_four, west_seed_five)
westsemi1 = playoff_series_sim(west18, west45)
print(westsemi1)
westsemi2 = playoff_series_sim(west27, west36)
print(westsemi2)
westfinals = playoff_series_sim(westsemi1, westsemi2)

C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


KeyboardInterrupt: 

In [ ]:
team_abbrev, elo_list, #first_round, second_round, conference_semifinals, finals, championship

In [ ]:
#Eastern playoffs
east18 = playoff_series_sim(east_seed_one, east_seed_eight)
east27 = playoff_series_sim(east_seed_two, east_seed_seven)
east36 = playoff_series_sim(east_seed_three, east_seed_six)
east45 = playoff_series_sim(east_seed_four, east_seed_five)
eastsemi1 = playoff_series_sim(east18, east45)
print(eastsemi1)
eastsemi2 = playoff_series_sim(east27, east36)
print(eastsemi2)
eastfinals = playoff_series_sim(eastsemi1, eastsemi2)

In [ ]:
print(westfinals)
print(eastfinals)

In [ ]:
#NBA Finals
champion = playoff_series_sim(westfinals, eastfinals)

In [ ]:
print(champion)

In [ ]:
#Create counting variable and list to place champions
# i = 1
# champions_list = []
# iterations = 50

# while i <= iterations:    
#     # Reset season to start
#     team_df = team_df_backup
    
#     # Do all the season steps
#     team_df = all_regular_season(team_df)
    
#     # Add win percentage
#     team_df['win_percentage'] = team_df['wins']/(team_df['wins']+team_df['losses'])
    
#     # Create conference specific dataframes
#     df_western = team_df[team_df['conference'] == 'western']
#     df_eastern = team_df[team_df['conference'] == 'eastern']
#     df_western = bracket_cleaner(df_western)
#     df_eastern = bracket_cleaner(df_eastern)
    
#     # Create conference seeds
#     west_seed_one = df_western.loc[0]
#     west_seed_two = df_western.loc[1]
#     west_seed_three = df_western.loc[2]
#     west_seed_four = df_western.loc[3]
#     west_seed_five = df_western.loc[4]
#     west_seed_six = df_western.loc[5]
#     west_seed_seven = df_western.loc[6]
#     west_seed_eight = play_in_round(df_western)
#     east_seed_one = df_eastern.loc[0]
#     east_seed_two = df_eastern.loc[1]
#     east_seed_three = df_eastern.loc[2]
#     east_seed_four = df_eastern.loc[3]
#     east_seed_five = df_eastern.loc[4]
#     east_seed_six = df_eastern.loc[5]
#     east_seed_seven = df_eastern.loc[6]
#     east_seed_eight = play_in_round(df_eastern)
   
#     # Simulate Western playoffs
#     west18 = playoff_series_sim(west_seed_one, west_seed_eight)
#     west27 = playoff_series_sim(west_seed_two, west_seed_seven)
#     west36 = playoff_series_sim(west_seed_three, west_seed_six)
#     west45 = playoff_series_sim(west_seed_four, west_seed_five)
#     westsemi1 = playoff_series_sim(west18, west45)
#     westsemi2 = playoff_series_sim(west27, west36)
#     westfinals = playoff_series_sim(westsemi1, westsemi2)

#     # Simulate Eastern playoffs
#     east18 = playoff_series_sim(east_seed_one, east_seed_eight)
#     east27 = playoff_series_sim(east_seed_two, east_seed_seven)
#     east36 = playoff_series_sim(east_seed_three, east_seed_six)
#     east45 = playoff_series_sim(east_seed_four, east_seed_five)
#     eastsemi1 = playoff_series_sim(east18, east45)
#     eastsemi2 = playoff_series_sim(east27, east36)
#     eastfinals = playoff_series_sim(eastsemi1, eastsemi2)

#     champion_winner = playoff_series_sim(westfinals, eastfinals)
#     champion = champion_winner['team_abbrev']

    
#     # Append champions list
#     champions_list.append(champion)
    
#     # Increase counter
#     i += 1

In [ ]:
# Get counts of champions
# Counter(champions_list)

In [ ]:
def game_simulation_regular_season(team_a, team_b, df):
    team_a = team_a
    team_b = team_b
    input_df = df
    ###
    #Get elo values from database
    ###
    elo_a = input_df['elo_list'][input_df['team_abbrev'] == team_a].values[0][-1]
    #print("elo a", elo_a)
    
    elo_b = input_df['elo_list'][input_df['team_abbrev'] == team_b].values[0][-1]
    #print("elo b", elo_b)
    
    # Run elo update and game winner functions
    new_elo_a, new_elo_b, game_winner_output = elo_update(elo_a, elo_b)
    
    # Update elo for first team
    input_df['elo_list'][input_df['team_abbrev'] == team_a].values[0].append(new_elo_a)
    
    # Update elo for second team
    input_df['elo_list'][input_df['team_abbrev'] == team_b].values[0].append(new_elo_b)
    
    # Add wins and losses to respective teams
    if game_winner_output == 1:
        input_df['wins'][input_df['team_abbrev'] == team_a] += 1
        input_df['losses'][input_df['team_abbrev'] == team_b] += 1
    elif game_winner_output == 2:
        input_df['wins'][input_df['team_abbrev'] == team_b] += 1
        input_df['losses'][input_df['team_abbrev'] == team_a] += 1
    else:
        pass
    return input_df

In [ ]:
# Define function to simulate all regular season games and return a final dataframe at the end of the regular season
def simulate_regular_season(df, schedule_df):
    input_df = df
    # Simulate the season
    for index, row in schedule_df.iterrows():
        game_simulation_regular_season(row['team_a'], row['team_b'], input_df)

    # Create certain column that will be needed later on
    input_df['elo_final_regular_season'] = [x[-1] for x in input_df['elo_list']]
    input_df['winning_percentage'] = input_df['wins']/(input_df['wins']+input_df['losses'])
    input_df['elo_into_playoffs'] = 0.5 * input_df['elo_value'] + 0.5 * input_df['elo_final_regular_season']

    # Create western conference and eastern conference dataframes
    western_df = input_df[input_df.conference == 'western'].sort_values(by=['winning_percentage', 'elo_final_regular_season'], ascending=False)
    western_df['seed'] = [x for x in range(1, len(western_df.team_abbrev)+1)]
    eastern_df = input_df[input_df.conference == 'eastern'].sort_values(by=['winning_percentage', 'elo_final_regular_season'], ascending=False)
    eastern_df['seed'] = [x for x in range(1, len(eastern_df.team_abbrev)+1)]

    return input_df, western_df, eastern_df

In [203]:
west_df_output

,team_abbrev,team_nickname,elo_basic,elo_playoff_full,wins,losses,conference,elo_value,elo_list,elo_final_regular_season,winning_percentage,elo_into_playoffs,seed
0,LAL,Lakers,1678,1710,56,15,western,1694.0,"[1694.0, 1703.71225632, 1712.27701006, 1719.13...",1731.149836,0.788732,1712.574918,1
1,LAC,Clippers,1636,1732,49,23,western,1684.0,"[1684.0, 1674.28774368, 1661.28309103, 1666.69...",1682.773814,0.680556,1683.386907,2
11,UTA,Jazz,1574,1584,47,25,western,1579.0,"[1579.0, 1588.92804546, 1599.11717896, 1592.26...",1611.018180,0.652778,1595.009090,3
7,DEN,Nuggets,1579,1638,46,27,western,1608.5,"[1608.5, 1616.65067539, 1605.75108174, 1612.31...",1586.544461,0.630137,1597.522230,4
8,OKC,Thunder,1608,1583,44,28,western,1595.5,"[1595.5, 1585.3108665, 1596.21046015, 1589.607...",1589.326211,0.611111,1592.413105,5
4,HOU,Rockets,1565,1701,43,29,western,1633.0,"[1633.0, 1622.43933347, 1615.00376332, 1622.65...",1609.085175,0.597222,1621.042587,6
9,DAL,Mavericks,1584,1643,42,33,western,1613.5,"[1613.5, 1624.06066653, 1610.01385154, 1597.22...",1569.664731,0.560000,1591.582365,7
18,SAC,Kings,1509,1512,34,38,western,1510.5,"[1510.5, 1519.79599983, 1510.48058368, 1523.26...",1561.720368,0.472222,1536.110184,8
14,POR,Trailblazers,1490,1569,33,41,western,1529.5,"[1529.5, 1538.78168146, 1531.78454805, 1524.13...",1541.883061,0.445946,1535.691530,9
10,NOP,Pelicans,1542,1611,32,40,western,1576.5,"[1576.5, 1566.57195454, 1579.57660719, 1586.91...",1566.414672,0.444444,1571.457336,10


In [119]:
team_df.columns

Index([u'team_abbrev', u'team_nickname', u'elo_basic', u'elo_playoff_full',
       u'wins', u'losses', u'conference', u'elo_value', u'elo_list',
       u'elo_final_regular_season', u'winning_percentage',
       u'elo_into_playoffs'],
      dtype='object')

In [197]:
appending_df = pd.DataFrame({u'team_abbrev':[], u'team_nickname':[], u'elo_basic':[], u'elo_playoff_full':[],
       u'wins':[], u'losses':[], u'conference':[], u'elo_value':[], u'elo_list':[],
       u'elo_final_regular_season':[], u'winning_percentage':[],
       u'elo_into_playoffs':[], 'seed':[]})
#team_df_output = team_df.copy(deep=True)
for i in range(5):
    team_df = original_team_object.copy(deep=True)
    team_df['elo_value'] = elo_weight_start_season * team_df['elo_basic'] + (1 - elo_weight_start_season) * team_df['elo_playoff_full']
    team_df['elo_list'] = [[x] for x in team_df.elo_value]
    team_df_output, west_df_output, east_df_output = simulate_regular_season(team_df, schedule)
    appending_df = appending_df.append(west_df_output.append(east_df_output))
    #team_df_output = team_df.copy(deep=True)
    

C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [198]:
west_regular_season_outcomes = pd.pivot_table(appending_df[['team_abbrev','seed', 'team_nickname']][appending_df.conference == 'western'].groupby(['team_abbrev', 'seed']).count().reset_index(), values='team_nickname', index=['team_abbrev'], columns=['seed'], aggfunc=np.sum)
west_regular_season_outcomes.reindex(west_regular_season_outcomes[1.0].sort_values(ascending=False).index)

seed,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0
team_abbrev,,,,,,,,,,,,,
LAL,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DAL,NaN,NaN,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
DEN,NaN,1.0,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HOU,NaN,NaN,1.0,2.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LAC,NaN,4.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,NaN,1.0,NaN,NaN
NOP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.0,NaN,NaN,NaN
OKC,NaN,NaN,NaN,NaN,1.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
PHX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,1.0


In [200]:
west_regular_season_outcomes

seed,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0
team_abbrev,,,,,,,,,,,,,
DAL,NaN,NaN,NaN,NaN,0.4,NaN,0.6,NaN,NaN,NaN,NaN,NaN,NaN
DEN,NaN,0.2,0.4,0.2,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HOU,NaN,NaN,0.2,0.4,NaN,0.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LAC,NaN,0.8,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LAL,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.6,0.2,NaN,0.2,NaN,NaN
NOP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.4,0.6,NaN,NaN,NaN
OKC,NaN,NaN,NaN,NaN,0.2,0.4,0.4,NaN,NaN,NaN,NaN,NaN,NaN
PHX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2,0.6,0.2


In [201]:
west_regular_season_outcomes/5

seed,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0
team_abbrev,,,,,,,,,,,,,
DAL,NaN,NaN,NaN,NaN,0.4,NaN,0.6,NaN,NaN,NaN,NaN,NaN,NaN
DEN,NaN,0.2,0.4,0.2,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HOU,NaN,NaN,0.2,0.4,NaN,0.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LAC,NaN,0.8,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LAL,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.6,0.2,NaN,0.2,NaN,NaN
NOP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.4,0.6,NaN,NaN,NaN
OKC,NaN,NaN,NaN,NaN,0.2,0.4,0.4,NaN,NaN,NaN,NaN,NaN,NaN
PHX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2,0.6,0.2


In [194]:
appending_df[(appending_df.conference == 'western') & (appending_df.team_abbrev == 'LAL')]

,conference,elo_basic,elo_final_regular_season,elo_into_playoffs,elo_list,elo_playoff_full,elo_value,losses,seed,team_abbrev,team_nickname,winning_percentage,wins
0,western,1678.0,1549.859697,1621.929849,"[1694.0, 1703.71225632, 1712.27701006, 1698.62...",1710.0,1694.0,32.0,4.0,LAL,Lakers,0.632184,55.0
0,western,1678.0,1508.077478,1601.038739,"[1694.0, 1703.71225632, 1712.27701006, 1698.62...",1710.0,1694.0,39.0,5.0,LAL,Lakers,0.589474,56.0
0,western,1678.0,1495.949082,1594.974541,"[1694.0, 1703.71225632, 1712.27701006, 1698.62...",1710.0,1694.0,45.0,6.0,LAL,Lakers,0.563107,58.0
0,western,1678.0,1486.599548,1590.299774,"[1694.0, 1703.71225632, 1712.27701006, 1698.62...",1710.0,1694.0,51.0,7.0,LAL,Lakers,0.540541,60.0
0,western,1678.0,1443.340921,1568.670461,"[1694.0, 1703.71225632, 1712.27701006, 1698.62...",1710.0,1694.0,59.0,8.0,LAL,Lakers,0.504202,60.0


In [205]:
west_df_output

,team_abbrev,team_nickname,elo_basic,elo_playoff_full,wins,losses,conference,elo_value,elo_list,elo_final_regular_season,winning_percentage,elo_into_playoffs,seed
0,LAL,Lakers,1678,1710,56,15,western,1694.0,"[1694.0, 1703.71225632, 1712.27701006, 1719.13...",1731.149836,0.788732,1712.574918,1
1,LAC,Clippers,1636,1732,49,23,western,1684.0,"[1684.0, 1674.28774368, 1661.28309103, 1666.69...",1682.773814,0.680556,1683.386907,2
11,UTA,Jazz,1574,1584,47,25,western,1579.0,"[1579.0, 1588.92804546, 1599.11717896, 1592.26...",1611.018180,0.652778,1595.009090,3
7,DEN,Nuggets,1579,1638,46,27,western,1608.5,"[1608.5, 1616.65067539, 1605.75108174, 1612.31...",1586.544461,0.630137,1597.522230,4
8,OKC,Thunder,1608,1583,44,28,western,1595.5,"[1595.5, 1585.3108665, 1596.21046015, 1589.607...",1589.326211,0.611111,1592.413105,5
4,HOU,Rockets,1565,1701,43,29,western,1633.0,"[1633.0, 1622.43933347, 1615.00376332, 1622.65...",1609.085175,0.597222,1621.042587,6
9,DAL,Mavericks,1584,1643,42,33,western,1613.5,"[1613.5, 1624.06066653, 1610.01385154, 1597.22...",1569.664731,0.560000,1591.582365,7
18,SAC,Kings,1509,1512,34,38,western,1510.5,"[1510.5, 1519.79599983, 1510.48058368, 1523.26...",1561.720368,0.472222,1536.110184,8
14,POR,Trailblazers,1490,1569,33,41,western,1529.5,"[1529.5, 1538.78168146, 1531.78454805, 1524.13...",1541.883061,0.445946,1535.691530,9
10,NOP,Pelicans,1542,1611,32,40,western,1576.5,"[1576.5, 1566.57195454, 1579.57660719, 1586.91...",1566.414672,0.444444,1571.457336,10


In [254]:
placement_df = {'team_abbrev':[x for x in west_df_output[west_df_output.seed >= 10].team_abbrev], 'conference':[x for x in west_df_output[west_df_output.seed >= 10].conference], 'placement':['DNQ' for x in west_df_output[west_df_output.seed >= 10].team_abbrev]}
placement_df

{'conference': ['western', 'western', 'western', 'western'],
 'placement': ['DNQ', 'DNQ', 'DNQ', 'DNQ'],
 'team_abbrev': ['NOP', 'MEM', 'PHX', 'SAS']}

In [255]:
# Play-in round function
def play_in_round(df):
    input_df = df
    input_df['ahead_wins'] = input_df['wins'].shift(1)
    input_df['ahead_losses'] = input_df['losses'].shift(1)
    input_df['games_back_of_ahead_team'] = ((input_df['ahead_wins'] - input_df['wins']) + (input_df['losses'] - input_df['ahead_losses']))/2
    eight = input_df[input_df.seed == 8]
    nine = input_df[input_df.seed == 9]
    for i in [eight, nine]:
        placement_df['conference'].append(i.conference.values[0])
        placement_df['placement'].append('PLAY_IN')
        placement_df['team_abbrev'].append(i.team_abbrev.values[0])
    if nine['games_back_of_ahead_team'].values > 4:
        return eight
    elif nine['games_back_of_ahead_team'].values <= 4:
        outcome1 = game_winner(eight['elo_list'].values[0][-1], nine['elo_list'].values[0][-1])
        #eight['elo_value'] = rating1_1
        #nine['elo_value'] = rating2_1
        if outcome1 == 1:
            return eight
        else:
            outcome2 = game_winner(eight['elo_list'].values[0][-1], nine['elo_list'].values[0][-1])
            #eight['elo_value'] = rating1_2
            #nine['elo_value'] = rating2_2
            if outcome2 == 1:
                return eight
            else:
                return nine

In [256]:
play_in_round(west_df_output)

,team_abbrev,team_nickname,elo_basic,elo_playoff_full,wins,losses,conference,elo_value,elo_list,elo_final_regular_season,winning_percentage,elo_into_playoffs,seed,ahead_wins,ahead_losses,games_back_of_ahead_team
18,SAC,Kings,1509,1512,34,38,western,1510.5,"[1510.5, 1519.79599983, 1510.48058368, 1523.26...",1561.720368,0.472222,1536.110184,8,42.0,33.0,6.5


In [257]:
placement_df

{'conference': ['western',
  'western',
  'western',
  'western',
  'western',
  'western'],
 'placement': ['DNQ', 'DNQ', 'DNQ', 'DNQ', 'PLAY_IN', 'PLAY_IN'],
 'team_abbrev': ['NOP', 'MEM', 'PHX', 'SAS', 'SAC', 'POR']}

In [ ]:
#Simulate playoff series
def playoff_series_sim(team_one, team_two):
    team1_wins = 0
    team2_wins = 0
    while team1_wins < 4 and team2_wins < 4:
        rating1, rating2, outcome = elo_update(team_one['elo_value'], team_two['elo_value'])
        team_one['elo_value'] = rating1
        team_two['elo_value'] = rating2
        if outcome == 1:
            team1_wins += 1
        elif outcome == 2:
            team2_wins += 1
    if team1_wins == 4:
        return team_one
    elif team2_wins == 4:
        return team_two
    else:
        return None